In [1]:
# 

applying deep learning on twitter’s sentiment analysis

*   Train Model - use keras to build and train a deep neural network model

*   Evaluate Model - measure the accuracy of the predictive model, and suggest further improvements


IMPORTING DATASET


In [3]:
!pip install pyLDAvis

In [4]:
from time import time
import pandas as pd
import numpy as np
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime
import pprint

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


In [5]:
#being able to read csv stored in google drive 
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Reading the dataset with no columns titles and with latin encoding 
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/tweetsClean.csv')
df.sample(3)



,Unnamed: 0,date,year,clean,url,tags,promote
5530024,5530024,2019-09-26,2019,surprised say hey everyone even peter schiff r...,,,
496076,496076,2018-06-04,2018,immvrse vr immvrs check immvrse pre ico june n...,,ImmVRs ImmVRse VR,
857630,857630,2019-01-23,2019,cfn media ’ exclusive interview aleafia health...,http://bit.ly/2FHPScz,weed crypto CBDpic.twitter.com/DxqRPs9G9r mari...,


In [7]:
# Checking if there is any missing value and datatype 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7080772 entries, 0 to 7080771
Data columns (total 7 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Unnamed: 0  int64 
 1   date        object
 2   year        int64 
 3   clean       object
 4   url         object
 5   tags        object
 6   promote     object
dtypes: int64(2), object(5)
memory usage: 378.2+ MB


In [8]:

# checking for null values, if any
df.isnull().sum()

Unnamed: 0        0
date              0
year              0
clean         15390
url               0
tags           9289
promote           0
dtype: int64

In [9]:
#ditching all row when text is null, as need text for analysis
df.dropna(how='any', inplace=True)

In [10]:
df.sample(3)

,Unnamed: 0,date,year,clean,url,tags,promote
2898350,2898350,2019-07-04,2019,stk possible play ogo hot cova ren btc edr mat...,https://t.co/gUkgcWLs1I,,
6011089,6011089,2019-10-14,2019,block number time utc miner antpool transactio...,,,
5898263,5898263,2019-10-10,2019,hope healing liver disease dog ebook bundle up...,https://t.co/xVHtjgaGmR,,


testing some embedding for deep learning

In [11]:
#bag of words = OPTION A - need to limit to 1000, else too long and gets 951k results
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=500, stop_words='english',analyzer='word', token_pattern=r'\w{1,}')
X = bow_vectorizer.fit_transform(df['clean'])
X.shape


(7056094, 500)

In [12]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

In [13]:
lda = LatentDirichletAllocation(n_components=10, verbose=1)
lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [14]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda.score(X))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda.perplexity(X))

# See model parameters
print(lda.get_params())

Log Likelihood:  -289905580.7340846
Perplexity:  150.10357872523622


TypeError: ignored

In [15]:
import pickle
# Save to file in the current working directory
pickleLDA = "/content/drive/MyDrive/Colab Notebooks/NLP/pickle_model_10.pkl"
with open(pickleLDA, 'wb') as file:
    pickle.dump(lda, file)
# # Load from file
# with open(pkl_filename, 'rb') as file:
#     pickle_model = pickle.load(file)
    
# # Calculate the accuracy score and predict target values
# score = pickle_model.score(Xtest, Ytest)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model.predict(Xtest)

In [16]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [17]:
pyLDAvis.sklearn.prepare(lda, X, bow_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.170793  0.124102       1        1  14.608334
1      0.189936  0.095202       2        1  13.971966
9     -0.203670 -0.102003       3        1  13.528849
2      0.146944  0.239111       4        1  10.186243
6      0.062806 -0.249478       5        1   9.199130
0      0.144888 -0.199844       6        1   9.176014
4     -0.177654  0.089790       7        1   8.621166
8     -0.236192 -0.023471       8        1   7.961532
3     -0.005332  0.017252       9        1   7.179442
7      0.249066  0.009340      10        1   5.567324, topic_info=        Term          Freq         Total Category  logprob  loglift
455      usd  1.322500e+06  1.322500e+06  Default  30.0000  30.0000
64       btc  5.054583e+06  5.054583e+06  Default  29.0000  29.0000
330    price  9.400290e+05  9.400290e+05  Default  28.0000  28.0000
184       gt  4.134640e+05  4.134640e+05  Default  27.0000  27.0000
489      xrp  8.676990e+05  8.676990e+05  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
489      xrp  7.677850e+04  8.676998e+05  Topic10  -3.7364   0.4633
423     time  4.637860e+04  3.806575e+05  Topic10  -4.2405   0.7832
330    price  5.402077e+04  9.400296e+05  Topic10  -4.0880   0.0317
461      utc  3.861583e+04  1.668613e+05  Topic10  -4.4237   1.4248
8    airdrop  3.758322e+04  2.649172e+05  Topic10  -4.4508   0.9354

[480 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.000448  account
0         2  0.457494  account
0         3  0.106609  account
0         5  0.000009  account
0         7  0.435435  account
...     ...       ...      ...
496      10  0.999367   ビットコイン
497       6  0.999984    プレゼント
498       6  0.606602     仮想通貨
498      10  0.393386     仮想通貨
499      10  0.999994        円

[1654 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 10, 3, 7, 1, 5, 9, 4, 8])

In [18]:
import random

#fetches 10 words from our vocabulary
for i in range(10):
    random_id = random.randint(0,len(bow_vectorizer.get_feature_names()))
    print(bow_vectorizer.get_feature_names()[random_id])

bytes
volume
rewards
emc
close
yobit
available
latest
users
investing


In [19]:
#find 10 words with the highest probability for the first topic
first_topic = lda.components_[0]
#sort the indexes according to probability values,
top_topic_words = first_topic.argsort()[-10:]
for i in top_topic_words:
    print(bow_vectorizer.get_feature_names()[i])

update
change
alert
プレゼント
average
current
volume
bitcoin
price
btc


In [20]:
#print the 10 words with highest probabilities for all the five topics
for i,topic in enumerate(lda.components_):
    print(f'Top 10 words for topic #{i}:')
    print([bow_vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['update', 'change', 'alert', 'プレゼント', 'average', 'current', 'volume', 'bitcoin', 'price', 'btc']


Top 10 words for topic #1:
['x', 'xlm', 'ada', 'bch', 'eos', 'trx', 'ltc', 'xrp', 'eth', 'btc']


Top 10 words for topic #2:
['eur', 'prices', 'ethereum', 'ltc', 'eth', 'bch', 'btc', 'xrp', 'bitcoin', 'usd']


Top 10 words for topic #3:
['crypto', 'free', 'utc', 'join', 'pic', 'airdrop', 'com', 'bitcoin', 'twitter', 'btc']


Top 10 words for topic #4:
['info', 'jobs', 'trade', 'blockchain', 'crypto', 'amp', 'trading', 'cryptocurrency', 'btc', 'bitcoin']


Top 10 words for topic #5:
['trading', 'ico', 'news', 'exchange', 'btc', 'ethereum', 'blockchain', 'cryptocurrency', 'crypto', 'bitcoin']


Top 10 words for topic #6:
['txs', 'sell', 'h', 'binance', 'price', 'k', 'buy', 'market', 'bitcoin', 'btc']


Top 10 words for topic #7:
['ltc', 'xrp', 'volume', 'transactions', '円', 'jpy', 'block', 'eth', 'btc', 'gt']


Top 10 words for topic #8:
['great', 'time', 'blockc

In [21]:
#add a column to the original data frame that will store the topic for the text
#This method will assign the probability of all the topics to each document
topic_values = lda.transform(X)
df['Topic'] = topic_values.argmax(axis=1)
df.sample(5)

,Unnamed: 0,date,year,clean,url,tags,promote,Topic
3520694,3520694,2019-07-23,2019,choose random followers win eth please follow ...,,freetoken freecoin Ethereum airdrop giveaway t...,,3
3883457,3883457,2019-08-03,2019,’ seen threads,https://t.co/SQALL3heTV,,,0
2489287,2489287,2019-06-24,2019,ae rapid long period increase h v users notifi...,https://t.co/yect2ma8nX https://t.co/mfcT6cyBb...,AE crypto coinalert blockchain altcoins binanc...,,1
4270152,4270152,2019-08-15,2019,august btc usd 😄 😋 investing fire ira k money ...,https://t.co/ksCOqyBwFU,money Savings crypto CapitalGain btc stock eth...,,4
5826809,5826809,2019-10-07,2019,btc この保ち合いの後 プレ企画 企画 仮想通貨 貰える プレゼント イーサリアム プレゼ...,,貰える 仮想通貨 airdrop プレ企画 BTC プレゼント イーサリアム 企画,,0


In [22]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/tweetsTopic.csv')